In [ ]:
!pip install transformers==4.43.2
!pip install bitsandbytes==0.43.3
!pip install accelerate==0.33.0

In [ ]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

HF_TOKEN = os.getenv("TOKEN")
login(token=HF_TOKEN)

In [ ]:
def load_model(base_model, torch_dtype=torch.bfloat16, use_quantization=False, quantization_config=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    attn_implementation = "eager"

    if use_quantization:
        if quantization_config is None:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch_dtype,
                bnb_4bit_use_double_quant=True,
            )
    else:
        bnb_config = None

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch_dtype,
        device_map="auto",
        attn_implementation=attn_implementation
    )

    if not use_quantization:
        model.to(device)

    return model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
device

In [ ]:
model = load_model(model_name, use_quantization=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = model.config

print(f"Memory Allocated: {round(torch.cuda.memory_allocated() / (1024 ** 2), 3)}GB")
print(f"Memory Reserved: {round(torch.cuda.memory_reserved() / (1024 ** 2), 3)}GB")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### 키워드 추출
: 20년도 이후 & 4단어 이상의 리뷰들을 도움돼요 개수가 많은 순으로 정렬한 후 상위 100개의 리뷰 데이터에 대해 적용

In [ ]:
import pandas as pd
df = pd.read_csv("kurly/20년도이후_4단어이상_help상위100.csv")

In [ ]:
#키워드 추출 프롬프트
system_prompt = '''
주어진 구문에서 제품의 주요 특성, 속성 또는 기능을 나타내는 명사 또는 구체적 표현을 키워드로 추출하세요. 감정적인 표현보다는 제품의 구체적인 속성에 중점을 두어야 합니다.
구문에 포함된 단어들을 분석하여 제품의 중요한 특성이나 기능을 정확히 표현하는 키워드를 도출합니다. 각 리뷰에서 중요한 제품 관련 키워드를 쉼표로 구분하여 추출하세요. 

- 키워드는 제품과 관련된 구체적인 명사 또는 기능을 나타내는 단어나 구입니다.
- **키워드가 동사나 형용사로 되어 있을 경우** 명사나 구 형태로 변환하여 추출하세요. 예를 들어, '간이 생각보다 쎄서'는 '간이 쎔'으로 변환하세요.
- 감정적인 표현(예: '좋아요', '실망스러워요')은 포함하지 마세요.
- 키워드는 쉼표로 구분된 목록으로 출력되어야 합니다.

예시:
- 구문: '정말 고기가 듬뿍 들었고'
  - 키워드: 고기, 양
- 구문: '배송이 너무 느려요'
  - 키워드: 배송
- 구문: '양도 많고 간단해서 좋네요'
  - 키워드: 양, 간편함
- 구문: '간이 생각보다 쎄서'
  - 키워드: 간이 쎔
- 구문: '포장이 꼼꼼하게 왔어요'
  - 키워드: 포장

모든 리뷰에서 이런 방식으로 키워드를 추출하세요.
'''

In [ ]:
#키워드 추출 함수
def extract_keywords_from_reviews(reviews):
    extracted_keywords = []

    for review in reviews:
        query = f'{review}\n키워드:'
        inputs = f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

        input_ids = tokenizer(inputs, return_tensors="pt").input_ids.to(device)
        outputs = model.generate(input_ids, max_new_tokens=500, do_sample=False)

        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if '키워드' in decoded_output:
            keyword_part = decoded_output.split('키워드:')[-1].strip()
            keywords = keyword_part.split('\n')[0]  
        else:
            keywords = "No keywords extracted"

        extracted_keywords.append(keywords)

    return extracted_keywords

In [ ]:
#추출한 키워드들을 'keywords'열에 추가
df['keywords'] = extract_keywords_from_reviews(df['cleaned_Kiwi_review'])

In [ ]:
#키워드가 포함된 데이터 생성
df.to_csv("kurly/help상위100_키워드포함.csv",index=False, encoding='utf-8-sig')

In [ ]:
#키워드들 중 중복되게 추출된 경우가 존재해서 중복을 없애는 과정
keywords_list = df['keywords'].tolist()

def remove_duplicates_from_list(keywords_list):
    all_unique_keywords = set()

    for keywords_str in keywords_list:
        keywords = keywords_str.split(', ')

        all_unique_keywords.update(keywords)

    unique_keywords_str = ', '.join(sorted(all_unique_keywords))

    return unique_keywords_str


unique_keywords = remove_duplicates_from_list(keywords_list)

In [ ]:
#추출된 키워드들을 리스트에 저장
unique_keywords_list = []

unique_keywords_list.extend([keyword.strip() for keyword in unique_keywords.split(',') if keyword.strip()])

#### 추출된 키워드
: 총 493개의 키워드

가격, 가성비, 가위, 가자미, 가족, 간, 간단 식사, 간단함, 간소한 조리법, 
간이, 간이 쎔, 간장, 간장, 간편, 간편함, 갈 빗살, 갈비, 갈비뼈, 갈비탕, 
갈빗대, 감자, 감자 칼국수면, 감자탕, 감칠맛, 감투, 강릉, 강비탕, 강추, 건강, 
건더기, 건데기, 겨울, 경상도 식, 경제적, 경험, 계란, 고객, 고급, 고기, 고기 분리, 
고기 양, 고명, 고사리, 고소, 고추, 고추가루, 고추기름, 고추장, 고춧가루, 곰탕, 
곱창, 구성, 국, 국간장, 국구릇, 국그릇, 국물, 국물 베이스, 국물 소스, 국물 양, 
국밥, 국산, 국수, 국수 베이스, 귀찮음, 기대 이상, 기름, 기름지기, 기분, 김, 
... ,
토종닭 속, 토종닭 육수, 토종닭 전문점, 토종닭 진한 국물, 토핑, 통마늘, 통뼈 감자탕, 
팁, 파, 팩, 팽이버섯, 펜션, 편리, 편리성, 편리함, 편안한 국, 편의성, 편함, 포만감,
포장, 포장 상태, 포장지, 퐉 풀, 표고버섯, 푸짐, 푸짐함, 푹 끓여지다, 풀무원, 
풀무원 만두, 품목, 품질, 프리미엄, 플라스틱, 피로, 핏물, 한 끼, 한 끼 식사용, 
한약재, 한약향, 한우, 할라피뇨, 할인, 해동, 해산물, 해장, 햄, 행복, 향, 현미햇반, 
호박, 혼밥, 혼자 먹기, 홍고추, 홍보, 후추, 후춧가루

### 카테고리 추출

#### 1. 맛/풍미
제품의 맛, 향, 풍미와 관련된 키워드들.

간, 간이, 간이 쎔, 감칠맛, 깊은 맛, 깊이, 고소, 매실 액, 매운맛이 적음, 매움, 매콤, 맛, 맛이 좋음, 맛있, 맛있게, 맛있는, 맛있다, 맛있어요, 맵기, 소금, 소스, 소스지, 진한, 진한 맛, 진함, 쫄깃, 쫄깃쫄깃, 쫄깃쫄깃한 식감, 쫄깃한 식감, 쫄깃함, 칼칼, 칼칼한 맛, 칼칼함, 텡글, 텡글텡글, 향, 한약향, 홍고추, 후추, 후춧가루

#### 2. 재료/구성
제품에 포함된 재료나 구성 요소와 관련된 키워드들.

갈 빗살, 갈비, 갈비뼈, 갈비탕, 갈빗대, 감자, 감자 칼국수면, 감자탕, 고기, 고기 분리, 고기 양, 고명, 고추, 고추가루, 고추기름, 곰탕, 곱창, 구성, 국, 국간장, 국수, 국물, 국물 베이스, 국물 소스, 국물 양, 국밥, 국산, 김, 김치, 김치찌개, 김칫국물, 김장김치, 나물, 냉동, 내장, 대파, 돼지 등뼈 척수, 돼지 머릿고기, 돼지고기, 된장, 된장찌개, 두부, 들기름, 들깨, 들깨 가루, 떡, 떡국, 떡국 떡, 떡만둣국, 라면 사리, 마늘, 만두, 만두국, 맛집, 메밀국수, 면, 면 사리, 무, 무나 콩나물, 문어, 미역, 미역국, 미원, 배추, 버섯, 부대찌개, 부추, 불고기, 비빔면, 새우, 설렁탕, 소고기, 순대, 순두부, 스팸, 쌀, 양, 양배추, 양지, 어묵, 우거지, 육개장, 육수, 육즙, 잡내, 전골, 전복, 조랭이 떡, 조미 가루, 조미료, 죽, 찌개, 찹쌀, 청국장, 청양고추, 토란, 해산물, 해장, 향, 호박

#### 3. 조리/보관 방법
제품의 조리법, 보관 상태 및 관련 방법에 대한 키워드들.

간단 식사, 간단함, 간소한 조리법, 간편, 간편함, 끓이기, 냉동 상태, 냉동 육개장, 냉동 제품, 냉동식품, 냉동실, 냉동실 보관, 데우기, 레시피, 리조트, 밀 키트, 밀가루, 밀키트, 밀키트류, 보관, 보관 상태, 조리, 조리법, 조절, 즉석, 해동

#### 4. 포장/배송
제품의 포장 상태, 배송, 보관과 관련된 키워드들.

배송, 포장, 포장 상태, 포장지, 봉투, 냉장, 냉장고, 냉장실, 아이스팩, 포장지

#### 5. 양/가격
제품의 양, 가성비 및 경제성과 관련된 키워드들.

가격, 가성비, 경제적, 대량, 양 조절, 양이, 양이 적당, 양이 적은 분, 양이 충분한 분, 일 인분 양, 중량, 푸짐, 푸짐함